## Collecting and analysing data from LinkedIn

In [1]:
#pragma nodebook off
#Use nodebook for better reproducibility https://github.com/uoa-eResearch/nodebook
%reload_ext nodebook.ipython
%nodebook disk phase4

<IPython.core.display.Javascript object>

In [ ]:
from linkedin_api import Linkedin
import pandas as pd  # tabular data
from tqdm.auto import tqdm # progress bars
tqdm.pandas()
import json
from pprint import pprint
import time

In [ ]:
secrets = json.load(open("secrets.json"))
api = Linkedin(secrets["username"], secrets["password"])

Sample search: https://www.linkedin.com/search/results/people/?geoUrn=%5B%22105490917%22%5D&origin=FACETED_SEARCH&pastCompany=%5B%221073%22%2C%221038%22%2C%223255299%22%2C%22397575%22%5D&sid=O0Q

KPMG New Zealand: https://www.linkedin.com/company/kpmg-new-zealand/: 1,083 employees, ~2500 past employees  
PwC New Zealand: https://www.linkedin.com/company/pwc-new-zealand/: 1,093 employees, ~1600 past employees  
Deloitte (Worldwide): https://www.linkedin.com/company/deloitte/: 363,547 employees ~1500 in NZ, ~4100 past employees in NZ  
EY (Worldwide): https://www.linkedin.com/company/ernstandyoung/: 319,968 employees, ~1200 in NZ, ~3400 past employees in NZ  

Summative past employees in NZ
~9700

375 people are currently working at one of the 4 and have past worked for one of the 4

In [ ]:
companies = [
    "3255299", # PwC
    "397575", # KPMG
    "1038", # Deloitte
    "1073" # EY
]
regions = [
    "105490917" # New Zealand
]
people = []
for company in tqdm(companies):
    people.extend(api.search_people(past_companies=[company], regions=regions))

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
len(people)

3939

In [ ]:
df = pd.DataFrame(people).drop_duplicates()
df

,urn_id,distance,public_id,tracking_id
0,ACoAABYLkwwBZlmZPuUAK9ZUaloP8tnTfb1tUeo,DISTANCE_2,daniela-dunn-950a45a3,369857292
1,ACoAAAL-g0sBdldYgucu1UpPGQa3tuwyEHyOt3k,DISTANCE_2,pieta-brown-8639a614,50234187
2,ACoAAADRaF4BRkwRZSQns6ix3T8mIcxdiEPyvxo,DISTANCE_2,sarahlocknz,13723742
3,ACoAAAR370cBo_Bgfx0jX4qvqnKeqTDz4BeY88Y,DISTANCE_2,yen-shih-94713421,74968903
4,ACoAAAUIBmwBMvRQptMG6uAstyPnHrJV65EJNpw,DISTANCE_2,henryhirsch,84412012
...,...,...,...,...
3926,ACoAAAJ__00BAF6a2ef0jG0sdSa5uQkUpQwZEbI,DISTANCE_3,ramunaidoo,41942861
3927,ACoAAB7LNS0Bl1usYfYGWfVfYdoCKZjeJdcsMlI,DISTANCE_3,yueyue-song-b51029125,516633901
3931,ACoAADGKPG0Bll5Jaczcxjxg7BRolLmp0v98xeo,DISTANCE_3,seohee-lee,831143021
3937,ACoAAANANKABuGRu00OWbNkB7ill6DrxXljB4Ts,DISTANCE_3,liz-libby-garvie-20032216,54539424


In [ ]:
df.distance.value_counts()

DISTANCE_3        3176
DISTANCE_2         595
DISTANCE_1           1
OUT_OF_NETWORK       1
Name: distance, dtype: int64

In [ ]:
pprint(api.get_profile('daniela-dunn-950a45a3'))

{'certifications': [],
 'education': [{'degreeName': 'BCom (Hons)',
                'description': 'Completed courses in computer networks, '
                               'enterprise resource planning systems, '
                               'security, and information systems research.  '
                               'Wrote a dissertation on multicast network cost '
                               'allocation.\n'
                               '\n'
                               'Achieved first class distinction.',
                'entityUrn': 'urn:li:fs_education:(ACoAABYLkwwBZlmZPuUAK9ZUaloP8tnTfb1tUeo,239453468)',
                'fieldOfStudy': 'Information Systems',
                'school': {'active': True,
                           'entityUrn': 'urn:li:fs_miniSchool:15518',
                           'logoUrl': 'https://media-exp1.licdn.com/dms/image/C560BAQEhkI7FEsIdTg/company-logo_',
                           'objectUrn': 'urn:li:school:15518',
                          

In [ ]:
print(f"To extract all {len(df)} profiles, at a rate of about 1 per second would take around {round(len(df) / 60 / 60, 2)} hours")

To extract all 3773 profiles, at a rate of about 1 per second would take around 1.05 hours


In [ ]:
results = []
e = None
for public_id in tqdm(df.public_id):
    try:
        results.append(api.get_profile(public_id))
        time.sleep(10)
    except Exception as e:
        print(e)
        break

  0%|          | 0/3773 [00:00<?, ?it/s]

In [ ]:
json.dump(people, open("people.json", "w"))
json.dump(results, open("profiles.json", "w"))

In [ ]:
nodes = {}
edges = {}
for r in tqdm(results):
    for i, e in enumerate(r["experience"]):
        companyName = e.get("companyName")
        if not companyName:
            pprint(e)
            continue
        if companyName not in nodes:
            nodes[companyName] = {
                "id": e.get("companyUrn"),
                "name": companyName,
                "geoLocationName": e.get("geoLocationName"),
                "locationName": e.get("locationName"),
                "industries": e.get("industries"),
                "companyLogoUrl": e.get("companyLogoUrl"),
                "val": 0
            }
        nodes[companyName]["val"] += 1
        if i < (len(r["experience"]) - 1):
            prevCompany = r["experience"][i + 1]
            prevCompanyName = prevCompany.get("companyName")
            if not prevCompanyName:
                pprint(prevCompany)
                continue
            edge_id = prevCompanyName + "_" + companyName
            if edge_id not in edges:
                edges[edge_id] = {
                    "source": prevCompanyName,
                    "target": companyName,
                    "val": 0
                }
            edges[edge_id]["val"] += 1

  0%|          | 0/3773 [00:00<?, ?it/s]

{'$anti_abuse_metadata': {'/companyName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/companyUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/description': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/entityUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/geoLocationName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/geoUrn': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/locationName': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
                          '/promotion': {'sourceUrns': {'com.linkedin.common.urn.MemberUrn': 'urn:li:member:347225297'}},
             

In [ ]:
node_df = pd.DataFrame(nodes.values())
pd.set_option('display.max_rows', 100)
node_df.sort_values(by="val", ascending=False).head(20)

,id,name,geoLocationName,locationName,industries,companyLogoUrl,val
1,urn:li:fs_miniCompany:3255299,PwC New Zealand,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,1210
104,urn:li:fs_miniCompany:397575,KPMG New Zealand,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,984
176,urn:li:fs_miniCompany:1073,EY,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C510BAQ...,795
151,urn:li:fs_miniCompany:1038,Deloitte,"Wellington, Wellington Region, New Zealand","Wellington, Wellington Region, New Zealand",None,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,450
38,urn:li:fs_miniCompany:1038,Deloitte New Zealand,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,436
52,None,PwC,"London, England, United Kingdom","London, England, United Kingdom",None,None,254
3,urn:li:fs_miniCompany:3691,The University of Auckland,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,219
54,urn:li:fs_miniCompany:270126,ASB Bank,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,194
26,urn:li:fs_miniCompany:7841,Bank of New Zealand,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,164
164,urn:li:fs_miniCompany:2437,ANZ,"Auckland, New Zealand","Auckland, New Zealand",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,163


In [ ]:
node_df[~pd.isna(node_df.geoLocationName) & ~pd.isna(node_df.locationName) & (node_df.geoLocationName!=node_df.locationName)]

,id,name,geoLocationName,locationName,industries,companyLogoUrl,val
3900,urn:li:fs_miniCompany:1038,Deloitte España,"Madrid y alrededores, España","Madrid Area, Spain",None,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1
3901,urn:li:fs_miniCompany:137323,Comwave,"Bogotá D.C., Colombia","Bogotá D.C. Area, Colombia",None,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1
3902,urn:li:fs_miniCompany:268836,Politécnico Grancolombiano,"Bogotá D.C., Colombia","Bogotá D.C. Area, Colombia",None,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,1
4061,None,"Advokátní kancelář JUDr. Sáša Navrátilová, MBA","Okres Brno-město, Česká republika","District Brno-City, Czech Republic",None,None,1
4388,urn:li:fs_miniCompany:1038,Deloitte France,"Région de Paris, France","Paris Area, France",None,https://media-exp1.licdn.com/dms/image/C4E0BAQ...,1
4544,urn:li:fs_miniCompany:801428,EF English First,中国 上海,"Shanghai City, China",None,https://media-exp1.licdn.com/dms/image/C4D0BAQ...,1
4611,urn:li:fs_miniCompany:163134,Coca-Cola FEMSA,"Ciudad de México y alrededores, México","Mexico City Area, Mexico",None,https://media-exp1.licdn.com/dms/image/C560BAQ...,2


In [ ]:
print(f"{sum(node_df.geoLocationName.str.contains('New Zealand', na=False))} companies in NZ out of {len(node_df)}")

2836 companies in NZ out of 6021


In [ ]:
edge_df = pd.DataFrame(edges.values())
edge_df.sort_values(by="val", ascending=False).head(20)

,source,target,val
18,PwC New Zealand,PwC New Zealand,332
130,KPMG New Zealand,KPMG New Zealand,210
437,EY,EY,181
42,Deloitte New Zealand,Deloitte New Zealand,94
418,Deloitte,Deloitte,67
59,PwC,PwC,56
627,Bank of New Zealand,Bank of New Zealand,53
642,Fonterra,Fonterra,51
356,ANZ,ANZ,50
76,ASB Bank,ASB Bank,48


In [ ]:
len(edges)

10448

In [ ]:
network = {
    "nodes": list(nodes.values()),
    "links": list(edges.values())
}
json.dump(network, open("network.json", "w"))